# Permutation and Crossjoins

Ideas and code for permutations and the efficient creation of cross joins / cartesian product.

In [1]:
import pandas as pd
import numpy as np
import functools

In [2]:
df1 = pd.DataFrame(data={'ABC':list('abcba')})

In [3]:
df2 = pd.DataFrame(data={'NUM':list(range(5))})

In [4]:
def add_series_index(df, idx_prefix='index_'):
    return df.assign(**{idx_prefix+df.columns[0]:range(df.shape[0])})

In [5]:
def _crossjoin_binary(df1, df2, join_on='__my_cross_index__'):
    """Computes the cartesian product of the two pd.DataFrames.

    Note, that the data frames are joined on a 'utility' column, which must exist in both data frames.
    """
    return pd.merge(df1, df2, on=join_on, copy=False)

In [6]:
def crossjoin(*dfs):
    """Compute cartesian product of all pd.DataFrame provided.
    """
    dfs = (pd.DataFrame(data=df).assign(__my_cross_index__=1) for df in dfs if not df.empty)
    try:
        initial = next(dfs)
    except StopIteration:
        return pd.DataFrame()
    else:
        return functools.reduce(_crossjoin_binary, dfs, initial).drop('__my_cross_index__', axis=1)

In [7]:
df1 = add_series_index(df1)
df2 = add_series_index(df2)
print(df1)
print(df2)

  ABC  index_ABC
0   a          0
1   b          1
2   c          2
3   b          3
4   a          4
   NUM  index_NUM
0    0          0
1    1          1
2    2          2
3    3          3
4    4          4


In [8]:
df_all = crossjoin(df1,df2)

In [9]:
df_all.sort_values(by=['index_ABC', 'index_NUM'], axis=0)

,ABC,index_ABC,NUM,index_NUM
0,a,0,0,0
1,a,0,1,1
2,a,0,2,2
3,a,0,3,3
4,a,0,4,4
5,b,1,0,0
6,b,1,1,1
7,b,1,2,2
8,b,1,3,3
9,b,1,4,4


In [10]:
df_all = crossjoin(df2,df1)
df_all.sort_values(by=['index_ABC', 'index_NUM'], axis=0)

,NUM,index_NUM,ABC,index_ABC
0,0,0,a,0
5,1,1,a,0
10,2,2,a,0
15,3,3,a,0
20,4,4,a,0
1,0,0,b,1
6,1,1,b,1
11,2,2,b,1
16,3,3,b,1
21,4,4,b,1


## using permutations

it does work but it's hard to make it work with arbitrary data types inside of it

https://docs.sympy.org/latest/modules/combinatorics/permutations.html#sympy.combinatorics.permutations.Permutation.inversions


In [100]:
import sympy as sy

In [101]:
from sympy.combinatorics import Permutation

### for simple numerical arrays

In [118]:
# there is two data frame with identical content, but with different order
data = list(range(10))
A = np.random.permutation(data)
B = np.random.permutation(data)
T = np.random.permutation(data)

In [119]:
print('A = \n{}'.format(A))
print('B = \n{}'.format(B))
print('T = \n{}'.format(T))

A = 
[6 0 4 1 9 2 5 3 8 7]
B = 
[4 1 9 0 7 8 3 5 6 2]
T = 
[6 2 4 5 3 9 0 1 7 8]


In [120]:
# get permutations
pA = Permutation(A)
print('pA = {}'.format(pA))
pB = Permutation(B)  #.from_sequence
print('pB = {}'.format(pB))
pT = Permutation(T)
print('pT = {}'.format(pT))

pA = (0 6 5 2 4 9 7 3 1)
pB = (0 4 7 5 8 6 3)(2 9)
pT = (0 6)(1 2 4 3 5 9 8 7)


In [121]:
# this should yield the 'normalized' order
B[(~pB).list()]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [122]:
# this should yield the order of T
B[(pT*~pB).list()]

array([6, 2, 4, 5, 3, 9, 0, 1, 7, 8])

In [123]:
# this should yield the order of T
A[(pT*~pA).list()]

array([6, 2, 4, 5, 3, 9, 0, 1, 7, 8])

In [125]:
assert(np.all(A[(pT*~pA).list()] == T))

## for arbitrary arrays

In [268]:
# create data with identical content but in different order

# A = pd.DataFrame(data={'x':list('ABC')})
# B = pd.DataFrame(data={'x':list('BAC')})
# T = pd.DataFrame(data={'x':list('CBA')})

# data = [(1,(4,2)),(5,(1,2)),(5,(1,4)),(111,(1,15))]
data = [(1,'A'),(5,'B'),(5,'AD'),(111,'x')]  
#data = list('ABCDEFG')

A = data.copy()
B = data.copy()
T = data.copy()
np.random.shuffle(A)
np.random.shuffle(B)
np.random.shuffle(T)

In [269]:
print('A = \n{}'.format(A))
print('B = \n{}'.format(B))
print('T = \n{}'.format(T))

A = 
[(111, 'x'), (1, 'A'), (5, 'AD'), (5, 'B')]
B = 
[(1, 'A'), (111, 'x'), (5, 'AD'), (5, 'B')]
T = 
[(5, 'B'), (5, 'AD'), (1, 'A'), (111, 'x')]


In [270]:
# get both permutations
#  need to construct from a sequence of tuples, not lists
#  I could get that from df.itertuples()
pA = Permutation.from_sequence(A)
print('pA = {}'.format(pA))
pB = Permutation.from_sequence(B)
print('pB = {}'.format(pB))
pT = Permutation.from_sequence(T)
print('pT = {}'.format(pT))

pA = (0 3 2 1)
pB = (1 3 2)
pT = (3)(0 2)


In [271]:
(~pB).list()

[0, 2, 3, 1]

In [272]:
# np arrays are convenient because of their [] indexing notation that allows to pass a list of indexes
npA = np.array(A)
npB = np.array(B)
npT = np.array(T)

In [273]:
# this should yield the 'normalized' order
npB[(~pB).list()]

array([['1', 'A'],
       ['5', 'AD'],
       ['5', 'B'],
       ['111', 'x']], dtype='<U21')

In [274]:
# this should yield the order of T
npB[(pT*~pB).list()]

array([['5', 'B'],
       ['5', 'AD'],
       ['1', 'A'],
       ['111', 'x']], dtype='<U21')

In [275]:
assert(np.all(npA[(pT*~pA).list()] == T))

In [276]:
# I can also do it with series - which is nice, because I do have series
dfB = pd.Series(data=B, name='B')
dfB.iloc[(pT*~pB).list()]

3      (5, B)
2     (5, AD)
0      (1, A)
1    (111, x)
Name: B, dtype: object